In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Viendo un poco los datos 

In [ ]:
#levantamos solo el train set 
df = pd.read_csv("/kaggle/input/osic-pulmonary-fibrosis-progression/train.csv")
given_test = pd.read_csv("/kaggle/input/osic-pulmonary-fibrosis-progression/test.csv")
sub = pd.read_csv("/kaggle/input/osic-pulmonary-fibrosis-progression/sample_submission.csv")
df.shape, given_test.shape, sub.shape

In [ ]:
df.head()

In [ ]:
given_test.head()

In [ ]:
# chequeamos si el test dato pertenece al train y vemos que efectivamente lo hace. 
set(given_test.Patient.values).issubset(set(df.Patient.values))

In [ ]:
# de qué forma deberíamos entregar el submision?--> 
sub.head()

Analisamos un poco más del submision sample que es el archivo el cual tenemos que predecir? 

In [ ]:
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']].drop(columns = ['Patient_Week'])

In [ ]:
#los pacientes del sub son los mismos que los de test. 
set(sub.Patient.unique()) - set(given_test.Patient.unique())

In [ ]:
#en sub hay semanas que no tengo en train ? 
#tomo ejemplo de un solo paciente 
id_ = sub.Patient.unique()[4]
shapesub = sub[sub.Patient == id_].shape
shapedf = df[df.Patient == id_].shape
print('hay un total de muestras: {} para paciente x en submision'.format(shapesub[0]))
print('hay un total de muestras: {} para paciente x en train'.format(shapedf[0]))


Entonces acá es donde esta el verdadero quilombo 
- Hay solo 10 semanas de datos para predecir 130 aprox ? 
- es un problema temporal, dado cierto tiempo de datos predecir próximo tiempo. 
- Se evaluan la efectividad solamente en las últimas 3 semanas pero hay que predecir todas. 

Algunas especificaciones

In [ ]:
#tipos de columna
df.dtypes

In [ ]:
#algunos valores estadísticos 
df.describe()

In [ ]:
#chequeamos si alguna columna es nula (creo que con un solo valor nan ya salta True: chequear)
df.isnull().any()

In [ ]:
#hay valores duplicados en el train? 
df.duplicated().any()

In [ ]:
cantidad_pacientes = df.Patient.nunique()
cantidad_sex = df['Sex'].value_counts()
cantidad_status = df['SmokingStatus'].value_counts()
print('Cantidad total de pacientes :' + str(cantidad_pacientes))
print(cantidad_sex)
print(cantidad_status)

## Preproceso de los datos: 
El preproceso de los datos debe aplicarse tanto al train como al test set. Por ende se realiza un preproceso general luego se divide los sets de interes. 
El submision set es lo que hay que predecir. Hay que chequear si se repiten semanas del submision en el train, no estaría bien predecir sobre datos de entrenamiento. 
Luego del procesado y las predicciones el submision hay que volverlo a acomodar para enviarlo.

In [ ]:
#este merge me parece al pedo pero lo meto por las dudas ya que así esta donde lo robe 
sub = sub.merge(given_test.drop('Weeks', axis=1), on="Patient")
df['WHERE'] = 'train'
given_test['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = df.append([given_test, sub]) #juntamos test y sub 

In [ ]:
data['Female'] = data.Sex.apply(lambda x: 1 if x == 'Female' else 0)
data['Male'] = data.Sex.apply(lambda x: 1 if x == 'Male' else 0)
data['ExSmoker'] = data.SmokingStatus.apply(lambda x: 1 if x == 'Ex-smoker' else 0)
data['NeverSmoked'] = data.SmokingStatus.apply(lambda x: 1 if x == 'Never smoked' else 0)
data['CurrentlySmokes'] = data.SmokingStatus.apply(lambda x: 1 if x == 'Currently smokes' else 0)
data.head()

In [ ]:
data['semana_min'] = data['Weeks']
data['semana_min'] = data.groupby('Patient')['semana_min'].transform('min')
base = data.loc[data.Weeks == data.semana_min]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)
base.head(20)
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data.Weeks - data.semana_min
data = data.astype({"base_week": float})

data.head()

In [ ]:
#Normalización de los datos. Las columnas a usar en el modelo son (week, Percent, Age, Female,Male,ExSmoker,NeverSmoked,CurrentlySmokes, min_FVC)
#Los nombres no son exactamente igual a los del flaco pero es lo mismo
data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
data['Percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )
data['Age'] = (data['Age'] - data['Age'].min() ) / ( data['Age'].max() - data['Age'].min() )
data['min_FVC'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
data.head()

In [ ]:
#El valor de week queda distinto al chabon no se porque

In [ ]:

#=============================#
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)
#============================#
def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)
#=============================#
def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss
#=================
def make_model(nh):
    z = L.Input((nh,), name="Patient")
    x = L.Dense(100, activation="relu", name="d1")(z)
    x = L.Dense(100, activation="relu", name="d2")(x)
    #x = L.Dense(100, activation="relu", name="d3")(x)
    p1 = L.Dense(3, activation="linear", name="p1")(x)
    p2 = L.Dense(3, activation="relu", name="p2")(x)
    preds = L.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])
    
    model = M.Model(z, preds, name="CNN")
    #model.compile(loss=qloss, optimizer="adam", metrics=[score])
    model.compile(loss=mloss(0.8), optimizer=tf.keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False), metrics=[score])
    return model